In [1]:
!/usr/bin/python3 -m pip install --upgrade pip
!pip install keras-rl2
!pip install gym

In [21]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from tensorflow import keras
ENV_NAME = 'TRIA-1D'

In [34]:
class TriaTempEnv(Env):
    def __init__(self):
        #actions we can take is on, off.
        self.action_space = Discrete(2)
        
        # temperature array
        self.observation_space = Box(low=np.array([-25.0]), high=np.array([120.0]))
        
        # set starting temperature
        self.state = 40.0 + random.uniform(-2,2) #.randint(-2,2)
        
        # set tria simulation duration
        self.tria_length= 20
        
    def step(self, action):
        # apply action
        self.state +=  1 if action == 1 else -1
        
        #reduce tria simulation length by 1 second
        self.tria_length -= 1
        
        #claculate reward
        if self.state >= 40 and self.state <= 90:
            reward = 1 
        elif self.state >= 50 and self.state <= 85: 
            reward = 2
        elif self.state >= 65 and self.state <= 80:
            reward = 3 
        else:
            reward = -3
        
        if self.tria_length <= 0:
            done = True
        else:
            done = False
            
        # apply temperature noise    
        self.state += random.uniform(-1,1) #.randint(-1,1)
        
        #set placeholder
        info = {}
        
        return self.state, reward, done, info
            
        
    def render(self):
        pass
    
    def reset(self):
        self.state = 40.0 + random.uniform(-2,2) #.randint(-2,2)
        
        self.tria_length= 20
        
        return self.state
        

In [35]:
env = TriaTempEnv()

In [36]:
episodes = 10
for episode in range (1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        # env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        print('Episode: {} score: {}'.format(episode, score))
    

Episode: 1 score: -3
Episode: 1 score: -2
Episode: 1 score: -5
Episode: 1 score: -8
Episode: 1 score: -11
Episode: 1 score: -10
Episode: 1 score: -13
Episode: 1 score: -16
Episode: 1 score: -19
Episode: 1 score: -18
Episode: 1 score: -21
Episode: 1 score: -20
Episode: 1 score: -19
Episode: 1 score: -18
Episode: 1 score: -21
Episode: 1 score: -24
Episode: 1 score: -27
Episode: 1 score: -26
Episode: 1 score: -29
Episode: 1 score: -32
Episode: 2 score: 1
Episode: 2 score: 2
Episode: 2 score: -1
Episode: 2 score: -4
Episode: 2 score: -7
Episode: 2 score: -10
Episode: 2 score: -13
Episode: 2 score: -12
Episode: 2 score: -11
Episode: 2 score: -10
Episode: 2 score: -9
Episode: 2 score: -12
Episode: 2 score: -11
Episode: 2 score: -10
Episode: 2 score: -9
Episode: 2 score: -8
Episode: 2 score: -7
Episode: 2 score: -6
Episode: 2 score: -5
Episode: 2 score: -4
Episode: 3 score: -3
Episode: 3 score: -6
Episode: 3 score: -5
Episode: 3 score: -4
Episode: 3 score: -3
Episode: 3 score: -2
Episode: 3 s

In [37]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [38]:
states = env.observation_space.shape
actions = env.action_space.n

In [39]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [40]:
del model

In [41]:
model = build_model(states, actions)

In [42]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 24)                48        
                                                                 
 dense_13 (Dense)            (None, 24)                600       
                                                                 
 dense_14 (Dense)            (None, 24)                600       
                                                                 
 dense_15 (Dense)            (None, 2)                 50        
                                                                 
Total params: 1,298
Trainable params: 1,298
Non-trainable params: 0
_________________________________________________________________


In [43]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [44]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
          nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [45]:
from tensorflow.keras.optimizers.legacy import Adam
#opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.1)
#Adam._name = 'hey'
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)  

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 634s 63ms/step - reward: 0.5124
500 episodes - episode_reward: 10.248 [-60.000, 20.000] - loss: 4.727 - mae: 6.292 - mean_q: 11.664

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 653s 65ms/step - reward: 0.5976
500 episodes - episode_reward: 11.952 [-56.000, 20.000] - loss: 5.835 - mae: 7.554 - mean_q: 13.904

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 532s 53ms/step - reward: 0.6132
500 episodes - episode_reward: 12.264 [-48.000, 20.000] - loss: 5.190 - mae: 7.476 - mean_q: 13.761

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 470s 47ms/step - reward: 0.4184
500 episodes - episode_reward: 8.368 [-60.000, 20.000] - loss: 4.066 - mae: 7.227 - mean_q: 13.195

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 341s 34ms/step - reward: 0.7648
do

In [46]:
scores = dqn.test(env,nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 20.000, steps: 20
Episode 2: reward: 20.000, steps: 20
Episode 3: reward: 20.000, steps: 20
Episode 4: reward: 20.000, steps: 20
Episode 5: reward: 16.000, steps: 20
Episode 6: reward: 20.000, steps: 20
Episode 7: reward: 20.000, steps: 20
Episode 8: reward: 12.000, steps: 20
Episode 9: reward: 20.000, steps: 20
Episode 10: reward: 20.000, steps: 20
Episode 11: reward: 12.000, steps: 20
Episode 12: reward: 16.000, steps: 20
Episode 13: reward: 20.000, steps: 20
Episode 14: reward: 16.000, steps: 20
Episode 15: reward: 20.000, steps: 20
Episode 16: reward: 20.000, steps: 20
Episode 17: reward: 20.000, steps: 20
Episode 18: reward: 20.000, steps: 20
Episode 19: reward: 20.000, steps: 20
Episode 20: reward: 20.000, steps: 20
Episode 21: reward: 8.000, steps: 20
Episode 22: reward: 16.000, steps: 20
Episode 23: reward: 20.000, steps: 20
Episode 24: reward: 16.000, steps: 20
Episode 25: reward: 16.000, steps: 20
Episode 26: reward: 16.000, ste